# Connect to Local GraphDB Environment 

In [120]:
!pip install typing-inspect==0.8.0 typing_extensions==4.5.0
!pip install pydantic -U
!pip install pydantic==1.10.11
!pip install langchain --upgrade
!pip install -U langchain-openai
!pip install -U neo4j
!pip install -U openai
!pip install -U tiktoken
!pip install -U graphdatascience


  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydantic-core 2.18.2 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
openai 1.23.6 requires typing-extensions<5,>=4.7, but you have typing-extensions 4.5.0 which is incompatible.
langflow 0.0.68 requires langchain<0.1.0,>=0.0.150, but you have langchain 0.1.16 which is incompatible.
langflow 0.0.68 requires openai<0.28.0,>=0.27.2, but you have openai 1.23.6 which is incompatible.
langflow 0.0.68 requires tiktoken<0.4.0,>=0.3.3, but you have tiktoken 0.6.0 which is incompatible.


  Using cached pydantic-2.7.1-py3-none-any.whl (409 kB)
  Using cached typing_extensions-4.11.0-py3-none-any.whl (34 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.11
    Uninstalling pydantic-1.10.11:
      Successfully uninstalled pydantic-1.10.11


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Chris\\anaconda3\\Lib\\site-packages\\~-dantic\\annotated_types.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



  Using cached pydantic-1.10.11-cp39-cp39-win_amd64.whl (2.2 MB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.7.1
    Uninstalling pydantic-2.7.1:
      Successfully uninstalled pydantic-2.7.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langflow 0.0.68 requires langchain<0.1.0,>=0.0.150, but you have langchain 0.1.16 which is incompatible.
langflow 0.0.68 requires openai<0.28.0,>=0.27.2, but you have openai 1.23.6 which is incompatible.
langflow 0.0.68 requires tiktoken<0.4.0,>=0.3.3, but you have tiktoken 0.6.0 which is incompatible.


  Attempting uninstall: langchain-openai
    Found existing installation: langchain-openai 0.1.3
    Uninstalling langchain-openai-0.1.3:
      Successfully uninstalled langchain-openai-0.1.3
     ---------------------------------------- 0.0/203.0 kB ? eta -:--:--
     --------------- ----------------------- 81.9/203.0 kB 2.3 MB/s eta 0:00:01
     ------------------------------------ - 194.6/203.0 kB 2.4 MB/s eta 0:00:01
     -------------------------------------- 203.0/203.0 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for neo4j: filename=neo4j-5.20.0-py3-none-a

In [2]:
import os
import langchain
from langchain.graphs import Neo4jGraph
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from neo4j import GraphDatabase
    
url = "bolt://localhost:7687"
username ="neo4j"
password = "password"
#os.environ['OPENAI_API_KEY'] = "<removed>"

graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

driver = GraphDatabase.driver(url, auth=(username,password))

graph.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = ChatOpenAI(temperature=0, model_name='gpt-4'),
    qa_llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo'), graph=graph, verbose=True,
)

In [135]:
print(driver.execute_query("MATCH (p:Patient) RETURN p LIMIT 1"))

EagerResult(records=[<Record p=<Node element_id='4:6b5a7670-0efe-41ee-b146-d26e52a9a983:0' labels=frozenset({'Patient'}) properties={'HEALTHCARE_EXPENSES': 50725.19, 'LAST': 'Langosh790', 'DOB': '1999-03-13', 'GENDER': 'F', 'ID': '2703570e-f634-d4cc-13db-2ae14e19ae59', 'FIRST': 'Norma469'}>>], summary=<neo4j._work.summary.ResultSummary object at 0x0000023C53EC34F0>, keys=['p'])


In [33]:
cypher_chain.invoke("How many Patients are there?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Patient) RETURN COUNT(p)
Full Context:
[{'COUNT(p)': 121}]

> Finished chain.


{'query': 'How many Patients are there?', 'result': 'There are 121 Patients.'}

In [30]:
cypher_chain.invoke("How many Providers are there?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Provider) RETURN COUNT(p)
Full Context:
[{'COUNT(p)': 279}]

> Finished chain.


{'query': 'How many Providers are there?',
 'result': 'There are 279 Providers.'}

In [32]:
cypher_chain.invoke("How many Encounters are there?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (e:Encounter) RETURN COUNT(e)
Full Context:
[{'COUNT(e)': 11987}]

> Finished chain.


{'query': 'How many Encounters are there?',
 'result': 'There are 11987 Encounters.'}

In [29]:
cypher_chain.invoke("What is the total base cost across all encounters?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (e:Encounter)
RETURN SUM(e.BASE_ENCOUNTER_COST) AS TotalBaseCost
Full Context:
[{'TotalBaseCost': 1241311.1900002102}]

> Finished chain.


{'query': 'What is the total base cost across all encounters?',
 'result': 'The total base cost across all encounters is 1241311.1900002102.'}

In [5]:
cypher_chain.invoke("Which provider treated the most encounters?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Provider)-[:Treated]->(e:Encounter)
RETURN p.NAME, COUNT(e) AS Encounters_Treated
ORDER BY Encounters_Treated DESC
LIMIT 1
Full Context:
[{'p.NAME': 'Rudolf736 Vandervort697', 'Encounters_Treated': 812}]

> Finished chain.


{'query': 'Which provider treated the most encounters?',
 'result': 'Rudolf736 Vandervort697 treated 812 encounters.'}

In [6]:
cypher_chain.invoke("What are the top 5 procedures performed by Rudolf736 Vandervort697")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Provider {NAME: "Rudolf736 Vandervort697"})-[:Treated]->(e:Encounter)-[:Had]->(pr:Procedure)
RETURN pr.DESCRIPTION, COUNT(*) AS count
ORDER BY count DESC
LIMIT 5
Full Context:
[{'pr.DESCRIPTION': 'Renal dialysis (procedure)', 'count': 566}, {'pr.DESCRIPTION': 'Assessment of health and social care needs (procedure)', 'count': 65}, {'pr.DESCRIPTION': 'Assessment of substance use (procedure)', 'count': 49}, {'pr.DESCRIPTION': 'Depression screening (procedure)', 'count': 49}, {'pr.DESCRIPTION': 'Medication Reconciliation (procedure)', 'count': 48}]

> Finished chain.


{'query': 'What are the top 5 procedures performed by Rudolf736 Vandervort697',
 'result': 'The top 5 procedures performed by Rudolf736 Vandervort697 are Renal dialysis, Assessment of health and social care needs, Assessment of substance use, Depression screening, and Medication Reconciliation.'}

In [7]:
cypher_chain.invoke("What is the name of the patient that needed the most care?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Patient)-[:Had]->(e:Encounter)
RETURN p.FIRST, p.LAST, COUNT(e) AS Care_Needed
ORDER BY Care_Needed DESC
LIMIT 1
Full Context:
[{'p.FIRST': 'Margart214', 'p.LAST': 'Bogisich202', 'Care_Needed': 754}]

> Finished chain.


{'query': 'What is the name of the patient that needed the most care?',
 'result': 'Margart214 Bogisich202'}

In [8]:
cypher_chain.invoke("Provide a list of Margart214 Bogisich202's conditions")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Patient {FIRST: 'Margart214', LAST: 'Bogisich202'})-[:Has]->(c:Condition)
RETURN c.DESCRIPTION
Full Context:
[{'c.DESCRIPTION': 'Severe anxiety (panic) (finding)'}, {'c.DESCRIPTION': 'Part-time employment (finding)'}, {'c.DESCRIPTION': 'Proteinuria due to type 2 diabetes mellitus (disorder)'}, {'c.DESCRIPTION': 'Not in labor force (finding)'}, {'c.DESCRIPTION': 'Social isolation (finding)'}, {'c.DESCRIPTION': 'Victim of intimate partner abuse (finding)'}, {'c.DESCRIPTION': 'Stress (finding)'}, {'c.DESCRIPTION': 'Viral sinusitis (disorder)'}, {'c.DESCRIPTION': 'Essential hypertension (disorder)'}, {'c.DESCRIPTION': 'Chronic kidney disease stage 2 (disorder)'}]

> Finished chain.


{'query': "Provide a list of Margart214 Bogisich202's conditions",
 'result': 'Severe anxiety (panic), Part-time employment, Proteinuria due to type 2 diabetes mellitus, Not in labor force, Social isolation, Victim of intimate partner abuse, Stress, Viral sinusitis, Essential hypertension, Chronic kidney disease stage 2.'}

In [9]:
cypher_chain.invoke("Provide a list of Luanne915 Veum823's allergies")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Patient {FIRST: 'Luanne915', LAST: 'Veum823'})-[:Has]->(a:Allergies) RETURN a.DESCRIPTION
Full Context:
[{'a.DESCRIPTION': 'Allergy to substance (finding)'}, {'a.DESCRIPTION': 'Mold (organism)'}, {'a.DESCRIPTION': 'House dust mite (organism)'}, {'a.DESCRIPTION': 'Animal dander (substance)'}, {'a.DESCRIPTION': 'Grass pollen (substance)'}, {'a.DESCRIPTION': 'Aspirin'}, {'a.DESCRIPTION': 'Soy bean'}]

> Finished chain.


{'query': "Provide a list of Luanne915 Veum823's allergies",
 'result': 'Luanne915 Veum823 is allergic to Mold, House dust mite, Animal dander, Grass pollen, Aspirin, and Soy bean.'}

In [10]:
cypher_chain.invoke("What were Margart214 Bogisich202's most recent observations")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Patient {FIRST: 'Margart214', LAST: 'Bogisich202'})-[:Had]->(e:Encounter)-[:Had]->(o:Observation)
RETURN o
ORDER BY o.DATE DESC
LIMIT 1
Full Context:
[{'o': {'DATE': neo4j.time.DateTime(2024, 3, 27, 20, 51, 40, 0, tzinfo=<UTC>), 'CODE': '74006-8', 'ENCOUNTER': '3c51732a-078e-e1d1-33bf-bc6d1bcb548c', 'DESCRIPTION': 'Weight difference [Mass difference] --pre dialysis - post dialysis', 'VALUE': '4.7'}}]

> Finished chain.


{'query': "What were Margart214 Bogisich202's most recent observations",
 'result': 'The most recent observation for Margart214 Bogisich202 was a Weight difference [Mass difference] of 4.7 pre dialysis - post dialysis on March 27, 2024 at 8:51:40 PM.'}

In [11]:
cypher_chain.invoke("Patients with which conditions are most likely to expire")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Patient)-[:Has]->(c:Condition) WHERE p.DOD IS NOT NULL RETURN c.DESCRIPTION, COUNT(*) AS frequency ORDER BY frequency DESC LIMIT 10;
Full Context:
[{'c.DESCRIPTION': 'Medication review due (situation)', 'frequency': 21}, {'c.DESCRIPTION': 'Stress (finding)', 'frequency': 18}, {'c.DESCRIPTION': 'Full-time employment (finding)', 'frequency': 18}, {'c.DESCRIPTION': 'Part-time employment (finding)', 'frequency': 17}, {'c.DESCRIPTION': 'Not in labor force (finding)', 'frequency': 16}, {'c.DESCRIPTION': 'Received higher education (finding)', 'frequency': 15}, {'c.DESCRIPTION': 'Reports of violence in the environment (finding)', 'frequency': 14}, {'c.DESCRIPTION': 'Social isolation (finding)', 'frequency': 14}, {'c.DESCRIPTION': 'Limited social contact (finding)', 'frequency': 13}, {'c.DESCRIPTION': 'Anemia (disorder)', 'frequency': 13}]

> Finished chain.


{'query': 'Patients with which conditions are most likely to expire',
 'result': 'Patients with Anemia (disorder) are most likely to expire.'}

# Use the GraphDataScience Library to add a KNN similarity graph to our database

In [138]:
from graphdatascience import GraphDataScience
username ="neo4j"
local_pass = "password"
# Use Neo4j URI and credentials according to your setup
gds = GraphDataScience("bolt://localhost:7687", auth=(username,local_pass))

In [139]:
# Use this code to create a graphdatascience graph in the db (only needed once)

node_projection = {"Patient": {"properties": ["HEALTHCARE_EXPENSES"]},
                   "Condition": {"properties": []},
                   "Encounter": {"properties": ["BASE_ENCOUNTER_COST"]},
                   "Medication": {"properties": ["BASE_COST"]},
                   "Provider": {"properties": []}}
relationship_projection = {"Has": {"orientation": "UNDIRECTED"},"Treated": {"orientation": "UNDIRECTED"},"Had": {"orientation": "UNDIRECTED"}}

G, result = gds.graph.project("patient-graph", node_projection, relationship_projection)

print(f"The projection took {result['projectMillis']} ms")

result = gds.fastRP.mutate(
    G,
    mutateProperty="embedding",
    randomSeed=29,
    embeddingDimension=16,
    iterationWeights=[0.8, 1, 1, 1],
)


print(f"Graph '{G.name()}' node count: {G.node_count()}")
print(f"Graph '{G.name()}' node labels: {G.node_labels()}")

# Run kNN and write back to db
result = gds.knn.write(
    G,
    topK=2,
    nodeProperties=["embedding"],
    randomSeed=29,
    concurrency=1,
    sampleRate=1.0,
    deltaThreshold=0.0,
    writeRelationshipType="SIMILAR",
    writeProperty="score",
)

print(f"Relationships produced: {result['relationshipsWritten']}")
print(f"Nodes compared: {result['nodesCompared']}")
print(f"Mean similarity: {result['similarityDistribution']['mean']}")

The projection took 319 ms
Graph 'patient-graph' node count: 12713
Graph 'patient-graph' node labels: ['Condition', 'Medication', 'Encounter', 'Patient', 'Provider']
Relationships produced: 25426
Nodes compared: 12713
Mean similarity: 0.9766138740370915


In [114]:
graph.refresh_schema()

In [31]:
result = cypher_chain.invoke("Which conditions are most similar")
print(result["result"])



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c1:Condition)-[r:SIMILAR]->(c2:Condition)
RETURN c1.DESCRIPTION, c2.DESCRIPTION, r.score
ORDER BY r.score DESC
LIMIT 10;
Full Context:
[{'c1.DESCRIPTION': 'Non-small cell carcinoma of lung  TNM stage 1 (disorder)', 'c2.DESCRIPTION': 'Non-small cell lung cancer (disorder)', 'r.score': 1.0}, {'c1.DESCRIPTION': 'Malignant tumor of colon', 'c2.DESCRIPTION': 'Recurrent rectal polyp', 'r.score': 1.0}, {'c1.DESCRIPTION': 'Recurrent rectal polyp', 'c2.DESCRIPTION': 'Malignant tumor of colon', 'r.score': 1.0}, {'c1.DESCRIPTION': 'Microalbuminuria due to type 2 diabetes mellitus (disorder)', 'c2.DESCRIPTION': 'Chronic kidney disease stage 2 (disorder)', 'r.score': 1.0}, {'c1.DESCRIPTION': 'Whiplash injury to neck', 'c2.DESCRIPTION': 'Injury of neck (disorder)', 'r.score': 1.0}, {'c1.DESCRIPTION': 'Lack of access to transportation (finding)', 'c2.DESCRIPTION': 'Transport problem (finding)', 'r.score': 1.0}, {'c1.DESCRIPTION': '

In [20]:
cypher_chain.invoke("Which medications are most common in patients with Prediabetes?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Patient)-[:Has]->(c:Condition {DESCRIPTION: 'Prediabetes'}), (p)-[:Taking]->(m:Medication)
RETURN m.DESCRIPTION, COUNT(*) AS frequency
ORDER BY frequency DESC
Full Context:
[{'m.DESCRIPTION': 'amLODIPine 2.5 MG Oral Tablet', 'frequency': 18}, {'m.DESCRIPTION': 'Hydrochlorothiazide 25 MG Oral Tablet', 'frequency': 17}, {'m.DESCRIPTION': 'Acetaminophen 325 MG Oral Tablet', 'frequency': 17}, {'m.DESCRIPTION': 'Nitroglycerin 0.4 MG/ACTUAT Mucosal Spray', 'frequency': 16}, {'m.DESCRIPTION': '24 HR metoprolol succinate 100 MG Extended Release Oral Tablet', 'frequency': 16}, {'m.DESCRIPTION': 'lisinopril 10 MG Oral Tablet', 'frequency': 16}, {'m.DESCRIPTION': 'Clopidogrel 75 MG Oral Tablet', 'frequency': 14}, {'m.DESCRIPTION': 'Naproxen sodium 220 MG Oral Tablet', 'frequency': 14}, {'m.DESCRIPTION': 'Simvastatin 20 MG Oral Tablet', 'frequency': 13}, {'m.DESCRIPTION': 'insulin isophane  human 70 UNT/ML / insulin  regular  

{'query': 'Which medications are most common in patients with Prediabetes?',
 'result': 'amLODIPine 2.5 MG Oral Tablet is the most common medication in patients with Prediabetes.'}

In [21]:
cypher_chain.invoke("Which 2 patients have the most similar medications and conditions?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p1:Patient)-[:Taking]->(m:Medication)<-[:Taking]-(p2:Patient), (p1)-[:Has]->(c:Condition)<-[:Has]-(p2)
RETURN p1.ID, p2.ID, COUNT(m) AS Shared_Medications, COUNT(c) AS Shared_Conditions
ORDER BY Shared_Medications DESC, Shared_Conditions DESC
LIMIT 2
Full Context:
[{'p1.ID': '0b595f34-417b-c1ca-6e47-5c9bc7d8f58e', 'p2.ID': 'fa5c2948-0731-7bb9-5b00-b6be2409cc99', 'Shared_Medications': 416, 'Shared_Conditions': 416}, {'p1.ID': 'fa5c2948-0731-7bb9-5b00-b6be2409cc99', 'p2.ID': '0b595f34-417b-c1ca-6e47-5c9bc7d8f58e', 'Shared_Medications': 416, 'Shared_Conditions': 416}]

> Finished chain.


{'query': 'Which 2 patients have the most similar medications and conditions?',
 'result': 'The patients with IDs 0b595f34-417b-c1ca-6e47-5c9bc7d8f58e and fa5c2948-0731-7bb9-5b00-b6be2409cc99 have the most similar medications and conditions.'}

In [22]:
cypher_chain.invoke("Summarize the medication and condition history of patient 0b595f34-417b-c1ca-6e47-5c9bc7d8f58e")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Patient {ID: '0b595f34-417b-c1ca-6e47-5c9bc7d8f58e'})-[:Taking]->(m:Medication), (p)-[:Has]->(c:Condition)
RETURN p.ID as Patient_ID, p.FIRST as First_Name, p.LAST as Last_Name, collect(distinct m.DESCRIPTION) as Medication_History, collect(distinct c.DESCRIPTION) as Condition_History
Full Context:
[{'Patient_ID': '0b595f34-417b-c1ca-6e47-5c9bc7d8f58e', 'First_Name': 'Korey682', 'Last_Name': 'Abbott774', 'Medication_History': ['Acetaminophen 325 MG Oral Tablet', '24 HR metoprolol succinate 100 MG Extended Release Oral Tablet', 'Clopidogrel 75 MG Oral Tablet', 'lisinopril 10 MG Oral Tablet', 'insulin isophane  human 70 UNT/ML / insulin  regular  human 30 UNT/ML Injectable Suspension [Humulin]', 'Naproxen sodium 220 MG Oral Tablet', 'Simvastatin 20 MG Oral Tablet', '1 ML tacrolimus 5 MG/ML Injection', '1 ML Epoetin Alfa 4000 UNT/ML Injection [Epogen]', '0.25 ML Leuprolide Acetate 30 MG/ML Prefilled Syringe', '1 ML DO

{'query': 'Summarize the medication and condition history of patient 0b595f34-417b-c1ca-6e47-5c9bc7d8f58e',
 'result': "The medication history of patient 0b595f34-417b-c1ca-6e47-5c9bc7d8f58e includes a variety of medications such as Acetaminophen, Metoprolol, Clopidogrel, Lisinopril, Insulin, Naproxen, Simvastatin, Tacrolimus, Epoetin Alfa, Leuprolide Acetate, DOCEtaxel, Metformin, Hydrochlorothiazide, Nitroglycerin, and Hydrocodone Bitartrate. The patient's condition history involves issues like hyperglycemia, chronic kidney disease, obesity, anemia, prediabetes, viral sinusitis, end-stage renal disease, proteinuria, osteoarthritis, diabetes mellitus type 2, metabolic syndrome, ischemic heart disease, hypertension, anxiety, and more."}

In [23]:
 cypher_chain.invoke("Summarize the medication and condition history of patient fa5c2948-0731-7bb9-5b00-b6be2409cc99")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Patient {ID: 'fa5c2948-0731-7bb9-5b00-b6be2409cc99'})-[:Taking]->(m:Medication), (p)-[:Has]->(c:Condition)
RETURN p.ID as Patient_ID, p.FIRST as First_Name, p.LAST as Last_Name, collect(distinct m.DESCRIPTION) as Medication_History, collect(distinct c.DESCRIPTION) as Condition_History
Full Context:
[{'Patient_ID': 'fa5c2948-0731-7bb9-5b00-b6be2409cc99', 'First_Name': 'Joaquin141', 'Last_Name': 'Roberts511', 'Medication_History': ['0.25 ML Leuprolide Acetate 30 MG/ML Prefilled Syringe', 'Nitroglycerin 0.4 MG/ACTUAT Mucosal Spray', 'Hydrochlorothiazide 25 MG Oral Tablet', 'Clopidogrel 75 MG Oral Tablet', 'lisinopril 10 MG Oral Tablet', 'insulin isophane  human 70 UNT/ML / insulin  regular  human 30 UNT/ML Injectable Suspension [Humulin]', 'amLODIPine 2.5 MG Oral Tablet', '1 ML DOCEtaxel 20 MG/ML Injection', 'PACLitaxel 100 MG Injection', '1 ML tacrolimus 5 MG/ML Injection', 'Naproxen sodium 220 MG Oral Tablet', 'Acet

{'query': 'Summarize the medication and condition history of patient fa5c2948-0731-7bb9-5b00-b6be2409cc99',
 'result': 'The patient with ID fa5c2948-0731-7bb9-5b00-b6be2409cc99 has a medication history that includes various medications such as Leuprolide Acetate, Nitroglycerin, Hydrochlorothiazide, Clopidogrel, lisinopril, insulin, amLODIPine, DOCEtaxel, PACLitaxel, tacrolimus, Naproxen sodium, Acetaminophen, metoprolol succinate, Amoxicillin, Clavulanate, Simvastatin, and Cisplatin. Their condition history involves multiple conditions like Carcinoma in situ of prostate, Chronic kidney disease, Abnormal findings in diagnostic imaging of the heart and coronary circulation, Acute bronchitis, Microalbuminuria due to type 2 diabetes mellitus, Sleep disorder, Ischemic heart disease, and others.'}

In [24]:
 cypher_chain.invoke("Which 2 providers have treated the most similar procedures?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p1:Provider)-[:Treated]->(e1:Encounter)-[:Had]->(proc1:Procedure), (p2:Provider)-[:Treated]->(e2:Encounter)-[:Had]->(proc2:Procedure)
WHERE proc1.DESCRIPTION = proc2.DESCRIPTION AND p1.NAME <> p2.NAME
RETURN p1.NAME, p2.NAME, COUNT(proc1) AS Similar_Procedures
ORDER BY Similar_Procedures DESC
LIMIT 2
Full Context:
[{'p1.NAME': 'Alvin56 Crona259', 'p2.NAME': 'Rudolf736 Vandervort697', 'Similar_Procedures': 357996}, {'p1.NAME': 'Rudolf736 Vandervort697', 'p2.NAME': 'Alvin56 Crona259', 'Similar_Procedures': 357996}]

> Finished chain.


{'query': 'Which 2 providers have treated the most similar procedures?',
 'result': 'Alvin56 Crona259 and Rudolf736 Vandervort697 have treated the most similar procedures.'}

In [25]:
 cypher_chain.invoke("What are the top 5 procedures performed by Rudolf736 Vandervort697")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Provider {NAME: "Rudolf736 Vandervort697"})-[:Treated]->(e:Encounter)-[:Had]->(pr:Procedure)
RETURN pr.DESCRIPTION, COUNT(*) AS count
ORDER BY count DESC
LIMIT 5
Full Context:
[{'pr.DESCRIPTION': 'Renal dialysis (procedure)', 'count': 566}, {'pr.DESCRIPTION': 'Assessment of health and social care needs (procedure)', 'count': 65}, {'pr.DESCRIPTION': 'Assessment of substance use (procedure)', 'count': 49}, {'pr.DESCRIPTION': 'Depression screening (procedure)', 'count': 49}, {'pr.DESCRIPTION': 'Medication Reconciliation (procedure)', 'count': 48}]

> Finished chain.


{'query': 'What are the top 5 procedures performed by Rudolf736 Vandervort697',
 'result': 'The top 5 procedures performed by Rudolf736 Vandervort697 are Renal dialysis (procedure), Assessment of health and social care needs (procedure), Assessment of substance use (procedure), Depression screening (procedure), and Medication Reconciliation (procedure).'}

In [26]:
 cypher_chain.invoke("What are the top 5 procedures performed by Alvin56 Crona259")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Provider {NAME: 'Alvin56 Crona259'})-[:Treated]->(e:Encounter)-[:Had]->(pr:Procedure)
RETURN pr.DESCRIPTION, COUNT(*) AS procedure_count
ORDER BY procedure_count DESC
LIMIT 5
Full Context:
[{'pr.DESCRIPTION': 'Renal dialysis (procedure)', 'procedure_count': 625}, {'pr.DESCRIPTION': 'Assessment of health and social care needs (procedure)', 'procedure_count': 17}, {'pr.DESCRIPTION': 'Depression screening (procedure)', 'procedure_count': 15}, {'pr.DESCRIPTION': 'Depression screening using Patient Health Questionnaire Two-Item score (procedure)', 'procedure_count': 15}, {'pr.DESCRIPTION': 'Assessment of substance use (procedure)', 'procedure_count': 11}]

> Finished chain.


{'query': 'What are the top 5 procedures performed by Alvin56 Crona259',
 'result': 'The top 5 procedures performed by Alvin56 Crona259 are Renal dialysis (procedure), Assessment of health and social care needs (procedure), Depression screening (procedure), Depression screening using Patient Health Questionnaire Two-Item score (procedure), and Assessment of substance use (procedure).'}